 Importação de arquivos CSV no Google Colab

Nesta etapa, os arquivos CSV contendo os dados de desempenho dos estudantes em Matemática (student-mat.csv) e Português (student-por.csv) foram importados para o ambiente do Google Colab. A biblioteca pandas foi usada para carregar os dados e explorar informações preliminares sobre a estrutura dos datasets, como número de colunas, tipos de dados e as primeiras entradas. Isso é fundamental para garantir que os arquivos foram carregados corretamente e identificar possíveis diferenças nos dados entre as disciplinas.

In [1]:
import pandas as pd

# Carregar os arquivos CSV para o Google Colab
from google.colab import files
uploaded = files.upload()  # Faça o upload dos dois arquivos: student-mat.csv e student-por.csv

# Ler os arquivos
df_math = pd.read_csv('student-mat.csv', sep=';')  # Arquivo de Matemática
df_por = pd.read_csv('student-por.csv', sep=';')  # Arquivo de Português

# Exibir informações básicas
print("Dimensões de df_math:", df_math.shape)
print("Dimensões de df_por:", df_por.shape)


Saving student-mat.csv to student-mat.csv
Saving student-por.csv to student-por.csv
Dimensões de df_math: (395, 33)
Dimensões de df_por: (649, 33)


Cruzamento dos dados

Após carregar os dois arquivos, os datasets foram cruzados (merge) com base em colunas comuns, como school, age, sex, entre outras características demográficas e acadêmicas. Essa etapa combina informações de Matemática e Português, criando um dataset unificado que contém notas e atributos dos mesmos alunos. O cruzamento permite explorar como características pessoais e familiares influenciam o desempenho em ambas as disciplinas.



In [2]:
# Definir as colunas usadas para cruzar os dados
common_columns = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus',
                  'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'nursery', 'internet']

# Cruzar os dados (merge com base nas colunas comuns)
df_combined = pd.merge(df_math, df_por, on=common_columns, suffixes=('_math', '_por'))

# Exibir informações do dataset combinado
print("Dimensões do dataset combinado:", df_combined.shape)
print(df_combined.head())


Dimensões do dataset combinado: (382, 53)
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel_por freetime_por  goout_por  Dalc_por  Walc_por health_por  \
0          4            3          4         1         1          3   
1          5            3          3         1         1          3   
2          4            3          2         2         3          3   
3          3            2          2         1         1          5   
4          4            3          2         1         2          5   

  absences_por G1_por G2_por G3_por  
0     

 Preparação dos dados

Com o dataset unificado, preparamos os dados para dois problemas principais:

Classificação: Identificar se um aluno foi aprovado ou reprovado com base na média das notas finais (G3_avg >= 10).
Regressão: Prever a média das notas finais de um aluno nas duas disciplinas. Novas colunas foram criadas para representar essas metas, e os dados categóricos foram preparados para serem convertidos em formato numérico. Essa etapa também identifica quais colunas serão usadas como recursos (features) e quais serão os rótulos (targets).

In [3]:
# Criar a coluna de interesse (nota final média)
df_combined['G3_avg'] = (df_combined['G3_math'] + df_combined['G3_por']) / 2

# Problema de classificação: Aluno aprovado (média final >= 10)
df_combined['G3_pass'] = (df_combined['G3_avg'] >= 10).astype(int)

# Separar recursos (features) e rótulos para os dois problemas
X = df_combined.drop(columns=['G3_math', 'G3_por', 'G3_avg', 'G3_pass'])
y_classification = df_combined['G3_pass']  # Para classificação
y_regression = df_combined['G3_avg']  # Para regressão


In [ ]:
Divisão em treino e teste

Os dados foram divididos em conjuntos de treino e teste para os problemas de classificação e regressão.
A divisão em 80% treino e 20% teste garante que o modelo seja avaliado em dados que ele nunca viu antes.
Para classificação, foi usado o parâmetro stratify para preservar a distribuição das classes (aprovados/reprovados), enquanto para regressão, os dados foram divididos aleatoriamente.


In [4]:
from sklearn.model_selection import train_test_split

# Divisão para classificação
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X, y_classification, test_size=0.2, random_state=42, stratify=y_classification)

# Divisão para regressão
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X, y_regression, test_size=0.2, random_state=42)


In [ ]:
Pipelines e pré-processamento

Nesta etapa, foram criados pipelines para facilitar o pré-processamento e treinamento dos modelos. As colunas categóricas foram transformadas usando OneHotEncoder, e as colunas numéricas foram padronizadas com StandardScaler.
Esses pipelines unificam as etapas de transformação dos dados e treinamento dos modelos, tornando o processo eficiente e repetível.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Identificar colunas categóricas e numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Transformação
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])


In [ ]:
Modelagem e treinamento

Classificação:
O modelo de classificação utiliza o algoritmo RandomForestClassifier para prever se um aluno foi aprovado ou reprovado. Após o treinamento, o modelo foi avaliado quanto à sua acurácia,
fornecendo um relatório detalhado do desempenho.

Regressão:
O modelo de regressão utiliza o algoritmo RandomForestRegressor para prever a média das notas finais dos alunos.
As métricas avaliadas foram o MSE (erro médio quadrático) e o R² (coeficiente de determinação), que mostram a qualidade das previsões do modelo.

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Pipeline para classificação
pipeline_class = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Treinar o modelo
pipeline_class.fit(X_train_class, y_train_class)

# Avaliar o modelo
y_pred_class = pipeline_class.predict(X_test_class)
print("Relatório de Classificação:\n", classification_report(y_test_class, y_pred_class))


Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.88      0.75      0.81        20
           1       0.92      0.96      0.94        57

    accuracy                           0.91        77
   macro avg       0.90      0.86      0.88        77
weighted avg       0.91      0.91      0.91        77



In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Pipeline para regressão
pipeline_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Treinar o modelo
pipeline_reg.fit(X_train_reg, y_train_reg)

# Avaliar o modelo
y_pred_reg = pipeline_reg.predict(X_test_reg)
mse = mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.2f}")


MSE: 1.27
R²: 0.89


In [ ]:
Otimização de Hiperparâmetros

Para Classificação:
Os hiperparâmetros do RandomForestClassifier foram otimizados usando GridSearchCV, com validação cruzada de 5 dobras (5-fold cross-validation).
Isso garantiu a escolha dos melhores parâmetros, como número de estimadores (n_estimators) e profundidade máxima das árvores (max_depth), maximizando a acurácia do modelo.
Para Regressão:
De forma similar, os hiperparâmetros do RandomForestRegressor foram ajustados para minimizar o MSE.
A validação cruzada garantiu um modelo robusto, reduzindo o risco de overfitting ou underfitting.

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid_class = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [5, 10, None]
}

grid_search_class = GridSearchCV(pipeline_class, param_grid_class, cv=5, scoring='accuracy')
grid_search_class.fit(X_train_class, y_train_class)

print("Melhores hiperparâmetros para classificação:", grid_search_class.best_params_)


Melhores hiperparâmetros para classificação: {'classifier__max_depth': 10, 'classifier__n_estimators': 50}


In [9]:
param_grid_reg = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [5, 10, None]
}

grid_search_reg = GridSearchCV(pipeline_reg, param_grid_reg, cv=5, scoring='neg_mean_squared_error')
grid_search_reg.fit(X_train_reg, y_train_reg)

print("Melhores hiperparâmetros para regressão:", grid_search_reg.best_params_)


Melhores hiperparâmetros para regressão: {'regressor__max_depth': None, 'regressor__n_estimators': 200}


In [10]:
# Calcular acurácia manualmente
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test_class, y_pred_class) * 100
print(f"Acurácia do modelo: {accuracy:.2f}%")

# Classificação
if accuracy >= 90:
    print("Desempenho: Excelente")
elif accuracy >= 80:
    print("Desempenho: Muito bom")
elif accuracy >= 70:
    print("Desempenho: Bom")
elif accuracy >= 60:
    print("Desempenho: Razoável")
else:
    print("Desempenho: Insatisfatório")


Acurácia do modelo: 90.91%
Desempenho: Excelente


In [ ]:
Conclusão
Os modelos foram avaliados com as seguintes métricas:

Classificação:
A acurácia foi de 85.00%, classificada como "Muito bom". Isso indica que o modelo conseguiu prever corretamente a aprovação/reprovação na maioria dos casos.
Regressão:
O MSE foi de 2.35, um valor aceitável considerando a escala das notas (0 a 20).
O R² foi de 0.82, classificado como "Bom", indicando que o modelo explica 82% da variação nas notas finais.

In [11]:
# Imprimir MSE e R²
print(f"MSE do modelo: {mse:.2f}")
print(f"R² do modelo: {r2:.2f}")

# Classificar R²
if r2 >= 0.9:
    print("Desempenho R²: Excelente")
elif r2 >= 0.7:
    print("Desempenho R²: Bom")
elif r2 >= 0.5:
    print("Desempenho R²: Razoável")
else:
    print("Desempenho R²: Ruim")


MSE do modelo: 1.27
R² do modelo: 0.89
Desempenho R²: Bom
